In [1]:
import $ivy.`org.scalanlp:breeze_2.11:0.13.2`
import $ivy.`org.apache.spark:spark-core_2.11:2.4.3`
import $ivy.`org.apache.spark:spark-sql_2.11:2.4.3`
import $ivy.`org.apache.spark:spark-mllib_2.11:2.4.3`
import $ivy.`org.apache.spark:spark-graphx_2.11:2.4.3`

import $ivy.$                                
// import $ivy.`org.jfree:jfreechart:1.5.0`

import $ivy.$                                       

import $ivy.$                                      

import $ivy.$                                        

import $ivy.$                                         
// import $ivy.`org.apache.spark:spark-streaming_2.11:2.4.3`
//import $ivy.`sh.almond::ammonite-spark:0.3.0`

In [3]:
import breeze.linalg.csvread
import breeze.linalg.{DenseMatrix => DM}
import breeze.linalg.{DenseVector => DV}
import breeze.linalg
import java.io._
import scala.io.Source
import scala.collection.mutable.ArrayBuffer
import scala.collection.mutable.ListBuffer
import scala.util.matching.Regex
import org.apache.spark.sql.{DataFrame => DF}
import org.apache.spark.sql.SparkSession
import org.apache.spark.SparkConf
import org.apache.spark.sql.functions._
import org.apache.spark.sql.Column
import org.apache.spark.sql.SaveMode
import org.apache.spark.sql.types._
import org.apache.spark.sql.catalyst.plans.{Inner, LeftOuter, Cross}
import org.apache.log4j.Logger
import org.apache.log4j.Level
Logger.getLogger("org").setLevel(Level.ERROR)

In [4]:
import org.apache.spark.ml.{Pipeline, PipelineModel} 
import org.apache.spark.ml.feature.VectorAssembler
import org.apache.spark.ml.linalg.Vectors

import org.apache.spark.ml.{Pipeline, PipelineModel} 

import org.apache.spark.ml.feature.VectorAssembler

import org.apache.spark.ml.linalg.Vectors

In [15]:
val folder = new File("csc")
val files = folder.listFiles
val trainFile = files.filter(_.getName.matches("""train.csv""")).head
val testFile = files.filter(_.getName.matches("""test.csv""")).head
val structureFile = files.filter(_.getName.matches("""structures.csv""")).head

folder: File = csc
files: Array[File] = Array(
  csc/.DS_Store,
  csc/dipole_moments.csv,
  csc/magnetic_shielding_tensors.csv,
  csc/mulliken_charges.csv,
  csc/potential_energy.csv,
  csc/sample_submission.csv,
  csc/scalar_coupling_contributions.csv,
  csc/structures.csv,
  csc/structures.zip,
  csc/test.csv,
  csc/train.csv
)
trainFile: File = csc/train.csv
testFile: File = csc/test.csv
structureFile: File = csc/structures.csv

In [9]:
val sc = new SparkConf
sc.setAppName("GBT")
sc.setMaster("local[*]")
val spark = SparkSession.builder().config(sc).getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties


sc: SparkConf = org.apache.spark.SparkConf@4b7ee362
res8_1: SparkConf = org.apache.spark.SparkConf@4b7ee362
res8_2: SparkConf = org.apache.spark.SparkConf@4b7ee362
spark: SparkSession = org.apache.spark.sql.SparkSession@744fc4a0

In [11]:
var trainSample = 1.0
var testSample = 1.0
var (rTrain, rValid) = (0.75, 0.25)

trainSample: Double = 1.0
testSample: Double = 1.0
numFolds: Int = 10
rTrain: Double = 0.75
rValid: Double = 0.25

In [17]:
val trainAll = spark.read.option("header", "true").csv(trainFile.getAbsolutePath)
val testAll = spark.read.option("header", "true").csv(testFile.getAbsolutePath)
val structure = spark.read.option("header", "true").csv(testFile.getAbsolutePath)

In [20]:
val oldTargetCol = trainAll.columns.filter(_.matches("""scalar.*""")).head
val targetCol = "label"

oldTargetCol: String = "scalar_coupling_constant"
targetCol: String = "label"

In [22]:
val seed = 12345L
val splits = trainAll.withColumnRenamed(oldTargetCol, targetCol).randomSplit(Array(0.75, 0.25), seed)
val (trData, valData) = (splits(0), splits(1))

trainingData.cache
validationData.cache

val teData = testAll.sample(false, testSample).cache

seed: Long = 12345L
splits: Array[org.apache.spark.sql.Dataset[org.apache.spark.sql.Row]] = Array(
  [id: string, molecule_name: string ... 4 more fields],
  [id: string, molecule_name: string ... 4 more fields]
)
trData: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [id: string, molecule_name: string ... 4 more fields]
valData: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [id: string, molecule_name: string ... 4 more fields]
res21_3: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [id: string, molecule_name: string ... 4 more fields]
res21_4: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [id: string, molecule_name: string ... 4 more fields]
teData: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [id: string, molecule_name: string ... 3 more fields]

In [23]:
trData.show(10)

+-------+----------------+------------+------------+----+---------+
|     id|   molecule_name|atom_index_0|atom_index_1|type|    label|
+-------+----------------+------------+------------+----+---------+
|      0|dsgdb9nsd_000001|           1|           0|1JHC|  84.8076|
|      1|dsgdb9nsd_000001|           1|           2|2JHH|  -11.257|
|     10|dsgdb9nsd_000002|           1|           0|1JHN|  32.6889|
|    100|dsgdb9nsd_000012|           3|           5|3JHH|  12.0536|
|  10000|dsgdb9nsd_000524|          14|           5|1JHC|   86.787|
| 100000|dsgdb9nsd_003735|           7|           8|2JHH| -11.7083|
|1000002|dsgdb9nsd_034940|          15|           5|2JHC|-0.342991|
|1000003|dsgdb9nsd_034940|          15|           6|2JHC| -1.57302|
|1000004|dsgdb9nsd_034940|          15|           7|1JHC|   104.79|
|1000005|dsgdb9nsd_034940|          15|           8|3JHC|  4.37176|
+-------+----------------+------------+------------+----+---------+
only showing top 10 rows



In [78]:
var converterList = new ListBuffer[DF => DF]()

converterList: ListBuffer[DataFrame => DataFrame] = ListBuffer()

In [79]:
def convertTargerCol(df:DF) = {
    df.withColumn(targetCol, col(targetCol).cast(DoubleType))
}
converterList += convertTargerCol

defined function convertTargerCol
res78_1: ListBuffer[DataFrame => DataFrame] = ListBuffer(<function1>)

In [80]:
def convertTypeCol(df:DF):DF = {
    Seq(1, 4).foldLeft(df)((d, x) => d.withColumn("t" + x.toString, col("type").substr(x, 1)))
}
converterList += convertTypeCol

defined function convertTypeCol
res79_1: ListBuffer[DataFrame => DataFrame] = ListBuffer(
  <function1>,
  <function1>
)

In [82]:
import org.apache.spark.ml.feature.{StandardScaler, Bucketizer, CountVectorizer, Binarizer, MaxAbsScaler, StringIndexer, MinMaxScaler, OneHotEncoderEstimator, QuantileDiscretizer}

val categoricalCols = Seq("t1", "t4", "atom_index_0", "atom_index_1")
val newCategoricalCols = categoricalCols.map(_ + "_c")
val stringIndexStages = categoricalCols.map(c => new StringIndexer().setInputCol(c).setOutputCol(c + "_c").fit(convertTypeCol(trData).select(c)))
def indexStringCols(df:DF) = {
    stringIndexStages.foldLeft(df)((d,x) => x.transform(d))
}

converterList += indexStringCols

import org.apache.spark.ml.feature.{StandardScaler, Bucketizer, CountVectorizer, Binarizer, MaxAbsScaler, StringIndexer, MinMaxScaler, OneHotEncoderEstimator, QuantileDiscretizer}


categoricalCols: Seq[String] = List("t1", "t4", "atom_index_0", "atom_index_1")
newCategoricalCols: Seq[String] = List(
  "t1_c",
  "t4_c",
  "atom_index_0_c",
  "atom_index_1_c"
)
stringIndexStages: Seq[org.apache.spark.ml.feature.StringIndexerModel] = List(
  strIdx_2d1520fd099d,
  strIdx_ead29560e609,
  strIdx_584453947d70,
  strIdx_dc9e133abd20
)
defined function indexStringCols
res81_5: ListBuffer[DataFrame => DataFrame] = ListBuffer(
  <function1>,
  <function1>,
  <function1>
)

In [83]:
def convertAtomIndexType(df:DF):DF = {
    df.columns.filter(_.matches("""atom_index_\d{1}""")).foldLeft(df)((d, x) => d.withColumn(x, col(x).cast(DoubleType)))
}
converterList += convertAtomIndexType

defined function convertAtomIndexType
res82_1: ListBuffer[DataFrame => DataFrame] = ListBuffer(
  <function1>,
  <function1>,
  <function1>,
  <function1>
)

In [84]:
def makeNewColumnsFromAtomIndex(df:DF) = {
    val icols = df.columns.filter(_.matches("""atom_index.*"""))
    df.withColumn("atom_index_diff", abs( col(icols(0)) - col(icols(1)) ))
}
converterList += makeNewColumnsFromAtomIndex

defined function makeNewColumnsFromAtomIndex
res83_1: ListBuffer[DataFrame => DataFrame] = ListBuffer(
  <function1>,
  <function1>,
  <function1>,
  <function1>,
  <function1>
)

In [85]:
converterList.foldLeft(trData)((d, x) => d.transform( data => x(data))).show()

+-------+----------------+------------+------------+----+---------+---+---+----+----+--------------+--------------+---------------+
|     id|   molecule_name|atom_index_0|atom_index_1|type|    label| t1| t4|t1_c|t4_c|atom_index_0_c|atom_index_1_c|atom_index_diff|
+-------+----------------+------------+------------+----+---------+---+---+----+----+--------------+--------------+---------------+
|      0|dsgdb9nsd_000001|         1.0|         0.0|1JHC|  84.8076|  1|  C| 2.0| 0.0|          25.0|           7.0|            1.0|
|      1|dsgdb9nsd_000001|         1.0|         2.0|2JHH|  -11.257|  2|  H| 1.0| 1.0|          25.0|           0.0|            1.0|
|     10|dsgdb9nsd_000002|         1.0|         0.0|1JHN|  32.6889|  1|  N| 2.0| 2.0|          25.0|           7.0|            1.0|
|    100|dsgdb9nsd_000012|         3.0|         5.0|3JHH|  12.0536|  3|  H| 0.0| 1.0|          22.0|           4.0|            2.0|
|  10000|dsgdb9nsd_000524|        14.0|         5.0|1JHC|   86.787|  1|  C| 

In [51]:
// structure.select("molecule_name").dropDuplicates().count
// res49: Long = 45772L

// trData.select("molecule_name").dropDuplicates().count + valData.select("molecule_name").dropDuplicates().count
// res50: Long = 169869L

In [86]:
def convertDF(df:DF, list:ListBuffer[DF => DF]) = {
    list.foldLeft(df)((d, x) => d.transform( data => x(data)))
}

defined function convertDF

In [87]:
val featureCols = convertDF(trData, converterList).columns.filterNot(_.matches("""(id|molecule_name|type|t1|t4)""")).filterNot(_==targetCol)

featureCols: Array[String] = Array(
  "atom_index_0",
  "atom_index_1",
  "t1_c",
  "t4_c",
  "atom_index_0_c",
  "atom_index_1_c",
  "atom_index_diff"
)

In [88]:
val featureCol = "features"
val assembler = new VectorAssembler().setInputCols(featureCols).setOutputCol(featureCol)

featureCol: String = "features"
assembler: VectorAssembler = vecAssembler_0d0b90620a13

In [101]:
import org.apache.spark.ml.regression.{GBTRegressor, GBTRegressionModel}
import org.apache.spark.ml.regression.{GeneralizedLinearRegression, GeneralizedLinearRegressionModel}
import org.apache.spark.ml.regression.{LinearRegression, LinearRegressionModel}
import org.apache.spark.ml.{Pipeline, PipelineModel}
import org.apache.spark.ml.evaluation.RegressionEvaluator
import org.apache.spark.ml.tuning.ParamGridBuilder
import org.apache.spark.ml.tuning.CrossValidator
import org.apache.spark.mllib.evaluation.RegressionMetrics

import org.apache.spark.ml.regression.{GBTRegressor, GBTRegressionModel}

import org.apache.spark.ml.regression.{GeneralizedLinearRegression, GeneralizedLinearRegressionModel}

import org.apache.spark.ml.regression.{LinearRegression, LinearRegressionModel}

import org.apache.spark.ml.{Pipeline, PipelineModel}

import org.apache.spark.ml.evaluation.RegressionEvaluator

import org.apache.spark.ml.tuning.ParamGridBuilder

import org.apache.spark.ml.tuning.CrossValidator

import org.apache.spark.mllib.evaluation.RegressionMetrics

In [105]:
// Params for GBT
val NumTrees = Seq(5, 7, 10)
val MaxBins = Seq(3, 5, 7)
val numFolds = 10
val MaxIter:Seq[Int] = Seq(10)
val MaxDepth:Seq[Int] = Seq(10)
// Params for Linear Regression
val MaxIterLR: Seq[Int] = Seq(1000, 100)
val RegParam: Seq[Double] = Seq(0.001, 0.003, 0.005, 0.01)
val TolLR: Seq[Double] = Seq(1e-6)
val ElasticNetParam: Seq[Double] = Seq(0.001)

NumTrees: Seq[Int] = List(5, 7, 10)
MaxBins: Seq[Int] = List(3, 5, 7)
numFolds: Int = 10
MaxIter: Seq[Int] = List(10)
MaxDepth: Seq[Int] = List(10)
MaxIterLR: Seq[Int] = List(1000, 100)
RegParam: Seq[Double] = List(0.001, 0.003, 0.005, 0.01)
TolLR: Seq[Double] = List(1.0E-6)
ElasticNetParam: Seq[Double] = List(0.001)

In [91]:
import spark.implicits._

import spark.implicits._

In [110]:
val model1 = new GBTRegressor().setFeaturesCol(featureCol).setLabelCol(targetCol)
val model2 = new GeneralizedLinearRegression().setFeaturesCol(featureCol).setLabelCol(targetCol).setFamily("gaussian").setLink("identity")

model1: GBTRegressor = gbtr_cda9de589652
model2: GeneralizedLinearRegression = glm_f269716c7c87

In [111]:
val pipeline1 = new Pipeline().setStages(Array(assembler, model1))
val pipeline2 = new Pipeline().setStages(Array(assembler, model2))

pipeline1: Pipeline = pipeline_657a6dc282be
pipeline2: Pipeline = pipeline_cbedad117006

In [112]:
val paramGrid1 = new ParamGridBuilder().
    addGrid(model1.maxIter, MaxIter).
    addGrid(model1.maxDepth, MaxDepth).
    addGrid(model1.maxBins, MaxBins).
    build()
val paramGrid12 = new ParamGridBuilder().
    addGrid(model2.maxIter, MaxIterLR).
    addGrid(model2.tol, TolLR).
    addGrid(model2.regParam, RegParam).
    build()

paramGrid1: Array[org.apache.spark.ml.param.ParamMap] = Array(
  {
	gbtr_cda9de589652-maxBins: 3,
	gbtr_cda9de589652-maxDepth: 10,
	gbtr_cda9de589652-maxIter: 10
},
  {
	gbtr_cda9de589652-maxBins: 5,
	gbtr_cda9de589652-maxDepth: 10,
	gbtr_cda9de589652-maxIter: 10
},
  {
	gbtr_cda9de589652-maxBins: 7,
	gbtr_cda9de589652-maxDepth: 10,
	gbtr_cda9de589652-maxIter: 10
}
)
paramGrid12: Array[org.apache.spark.ml.param.ParamMap] = Array(
  {
	glm_f269716c7c87-maxIter: 1000,
	glm_f269716c7c87-regParam: 0.001,
	glm_f269716c7c87-tol: 1.0E-6
},
  {
	glm_f269716c7c87-maxIter: 100,
	glm_f269716c7c87-regParam: 0.001,
	glm_f269716c7c87-tol: 1.0E-6
},
  {
	glm_f269716c7c87-maxIter: 1000,
	glm_f269716c7c87-regParam: 0.003,
	glm_f269716c7c87-tol: 1.0E-6
},
  {
	glm_f269716c7c87-maxIter: 100,
	glm_f269716c7c87-regParam: 0.003,
	glm_f269716c7c87-tol: 1.0E-6
},
  {
	glm_f269716c7c87-maxIter: 1000,
	glm_f269716c7c87-regParam: 0.005,
	glm_f269716c7c87-tol: 1.0E-6
},
  {
	glm_f269716c7c87-maxIter: 100,
	glm_f2

In [113]:
val cv2 = new CrossValidator().
    setEstimator(pipeline).
    setEvaluator(new RegressionEvaluator).
    setEstimatorParamMaps(paramGrid).
    setNumFolds(numFolds)

cv2: CrossValidator = cv_88dd72f45199

In [114]:
val cvModel2 = cv2.fit(convertDF(trData, converterList))